In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AlignmentCalculator
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7442757197278366998"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [3]:
video_path = os.path.abspath("./tmp/video/7442757197278366998.mp4")
audio_path = os.path.abspath("./tmp/video/7442757197278366998.wav")

transcript = """
The truth is plain to see.
If you want freedom, take pride in your country.
If you want democracy, hold on to your sovereignty.
And if you want peace, love your nation.
Wise leaders always put the good of their own people
and their own country first.
The future does not belong to globalists.
The future belongs to patriots.
The future belongs to sovereign and independent nations
who protect their citizens, respect their neighbors,
and honor the differences that make each country special and unique.
"""

sentiment_results = {
    'positive': 0.5434603095054626,
    'neutral': 0.0,
    'negative': 0.45653969049453735
 }
description = '𝐕𝐢𝐢𝐭𝐨𝐫𝐮𝐥 𝐍𝐚ț𝐢𝐮𝐧𝐢𝐥𝐨𝐫 𝐒𝐮𝐯𝐞𝐫𝐚𝐧𝐞: 𝐌𝐞𝐬𝐚𝐣𝐮𝐥 𝐥𝐮𝐢 𝐃𝐨𝐧𝐚𝐥𝐝 𝐓𝐫𝐮𝐦𝐩 𝐝𝐞𝐬𝐩𝐫𝐞 𝐂ă𝐥𝐢𝐧 𝐆𝐞𝐨𝐫𝐠𝐞𝐬𝐜𝐮 #DonaldTrump #CălinGeorgescu #Suveranitate #Libertate #Democrație #Patriotism #Pace #NațiuniSuverane #LideriÎnțelepți #BinelePoporului #ViitorulPatrioților #MândrieNațională #ProtecțiaCetățenilor #RespectNațiuni #Independență '

In [4]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)
analysis_result_json

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7442757197278366998.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video presents a speech by Donald Trump, translated into Romanian, emphasizing themes of freedom, democracy, sovereignty, and patriotism. It highlights the importance of national pride, protecting citizens, and respecting neighbors. The speech contrasts 'patriots' with 'globalists,' asserting that the future belongs to sovereign nations and their leaders who prioritize their people and countries. The video also features a separate image of Că

{'summary': "The video presents a speech by Donald Trump, translated into Romanian, emphasizing themes of freedom, democracy, sovereignty, and patriotism. It highlights the importance of national pride, protecting citizens, and respecting neighbors. The speech contrasts 'patriots' with 'globalists,' asserting that the future belongs to sovereign nations and their leaders who prioritize their people and countries. The video also features a separate image of Călin Georgescu, implying a connection or endorsement.",
 'identified_subjects': [{'subject': 'donald trump', 'stance': 0.8},
  {'subject': 'calin georgescu', 'stance': 0.7},
  {'subject': 'patriotism', 'stance': 0.9},
  {'subject': 'globalists', 'stance': -0.8},
  {'subject': 'sovereignty', 'stance': 0.9},
  {'subject': 'democracy', 'stance': 0.7},
  {'subject': 'freedom', 'stance': 0.8},
  {'subject': 'peace', 'stance': 0.7}],
 'overall_alignment': 0.65}

In [5]:
identified_subjects = analysis_result_json["identified_subjects"]
llm_overall_alignment = analysis_result_json["overall_alignment"]
final_alignment, deterministic_alignment, alignment_conflict = AlignmentCalculator.calculate(identified_subjects, llm_overall_alignment, alpha=0.25)
print(f"llm_overall_alignment: {llm_overall_alignment} \nfinal_alignment: {final_alignment} \ndeterministic_alignment: {deterministic_alignment} \nalignment_conflict: {alignment_conflict}")

llm_overall_alignment: 0.65 
final_alignment: -0.3164874551971326 
deterministic_alignment: -0.3164874551971326 
alignment_conflict: 0.9664874551971326
